# View Support

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

25/06/22 04:47:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## Load Two Months of NYC Taxi/Limousine Trip Data

This notebook uses the New York City Taxi and Limousine Commission Trip Record Data available on the AWS Open Data Registry. This contains data of trips taken by taxis and for-hire vehicles in New York City. This data is stored in an iceberg table called `taxis`.

To be able to rerun the notebook several times, let's drop the table and the views if they exist to start fresh.

In [2]:
%%sql

CREATE DATABASE IF NOT EXISTS nyc.taxis;

++
||
++
++

In [3]:
%%sql

DROP TABLE IF EXISTS nyc.taxis


++
||
++
++

In [4]:
%%sql

DROP VIEW IF EXISTS nyc.long_distances

++
||
++
++

In [5]:
%%sql

DROP VIEW IF EXISTS nyc.negative_amounts

++
||
++
++

## Create the table

In [7]:
%%sql

CREATE TABLE nyc.taxis (
    VendorID              bigint,
    tpep_pickup_datetime  timestamp,
    tpep_dropoff_datetime timestamp,
    passenger_count       double,
    trip_distance         double,
    RatecodeID            double,
    store_and_fwd_flag    string,
    PULocationID          bigint,
    DOLocationID          bigint,
    payment_type          bigint,
    fare_amount           double,
    extra                 double,
    mta_tax               double,
    tip_amount            double,
    tolls_amount          double,
    improvement_surcharge double,
    total_amount          double,
    congestion_surcharge  double,
    airport_fee           double
)
USING iceberg
PARTITIONED BY (days(tpep_pickup_datetime))

++
||
++
++

## Write a month of data

In [8]:
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2022-01.parquet")
df.writeTo("nyc.taxis").append()

In [9]:
%%sql

SELECT * FROM nyc.taxis

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
2,2022-01-09 09:24:06,2022-01-09 09:45:35,1.0,9.81,1.0,N,138,244,2,29.0,0.0,0.5,0.0,6.55,0.3,37.6,0.0,1.25
2,2022-01-09 11:27:14,2022-01-09 11:47:43,1.0,9.01,1.0,N,138,230,2,27.5,0.0,0.5,0.0,6.55,0.3,38.6,2.5,1.25
2,2022-01-09 13:14:53,2022-01-09 13:23:52,1.0,3.52,1.0,N,138,7,1,12.0,0.0,0.5,2.81,0.0,0.3,16.86,0.0,1.25
2,2022-01-09 13:51:42,2022-01-09 14:16:59,1.0,10.1,1.0,N,138,162,2,29.0,0.5,0.5,0.0,6.55,0.3,40.6,2.5,1.25
2,2022-01-09 15:09:41,2022-01-09 15:38:33,1.0,11.76,1.0,N,138,181,2,35.5,0.5,0.5,0.0,0.0,0.3,38.05,0.0,1.25
2,2022-01-09 17:05:34,2022-01-09 17:36:28,1.0,12.86,1.0,N,138,37,2,36.0,0.5,0.5,0.0,0.0,0.3,38.55,0.0,1.25
2,2022-01-09 00:02:12,2022-01-09 00:11:44,1.0,2.43,1.0,N,48,90,2,9.5,0.5,0.5,0.0,0.0,0.3,13.3,2.5,0.0
2,2022-01-09 00:00:18,2022-01-09 00:12:53,1.0,4.92,1.0,N,114,75,1,15.0,0.5,0.5,3.76,0.0,0.3,22.56,2.5,0.0
2,2022-01-09 00:00:46,2022-01-09 00:13:20,1.0,3.04,1.0,N,140,246,1,11.5,0.5,0.5,2.5,0.0,0.3,17.8,2.5,0.0
2,2022-01-09 00:00:44,2022-01-09 00:05:42,1.0,1.37,1.0,N,141,162,1,6.0,0.5,0.5,1.2,0.0,0.3,11.0,2.5,0.0


## Create a view

Let's create an Iceberg view to look at the longest distances travelled and the total amount of the trips.

In [11]:
%%sql

CREATE VIEW nyc.long_distances (
    vendor_id COMMENT 'Vendor ID',
    pickup_date,
    dropoff_date,
    distance COMMENT 'Trip Distance',
    total COMMENT 'Total amount'
)
AS SELECT 
  VendorID,
  tpep_pickup_datetime,
  tpep_dropoff_datetime,
  trip_distance,
  total_amount
FROM nyc.taxis
ORDER BY trip_distance

++
||
++
++

In [12]:
%%sql

SELECT * FROM nyc.long_distances

vendor_id,pickup_date,dropoff_date,distance,total
1,2022-01-16 01:00:55,2022-01-16 01:21:30,0.0,38.98
2,2022-01-21 02:21:34,2022-01-21 02:21:42,0.0,62.8
2,2022-01-19 06:08:05,2022-01-19 06:08:08,0.0,3.3
2,2022-01-21 06:43:11,2022-01-21 06:44:37,0.0,18.6
1,2022-01-20 03:03:52,2022-01-20 03:04:22,0.0,40.35
1,2022-01-21 00:27:47,2022-01-21 01:00:56,0.0,42.0
2,2022-01-19 08:39:19,2022-01-19 08:39:21,0.0,66.96
2,2022-01-21 02:35:03,2022-01-21 02:35:20,0.0,27.36
1,2022-01-10 06:18:18,2022-01-10 06:25:42,0.0,11.6
2,2022-01-21 00:18:35,2022-01-21 00:19:10,0.0,65.3


### Update View to order results differently

The output isn't as helpful as imagined, so let's update the view and change the order of columns and the ordering of the results.

In [13]:
%%sql

CREATE OR REPLACE VIEW nyc.long_distances (
    distance COMMENT 'Trip Distance',
    total COMMENT 'Total amount',
    vendor_id COMMENT 'Vendor ID',
    pickup_date,
    dropoff_date
)
AS SELECT 
    trip_distance,
    total_amount,
    VendorID,
    tpep_pickup_datetime,
    tpep_dropoff_datetime
FROM nyc.taxis
WHERE trip_distance > 35
ORDER BY total_amount, trip_distance

++
||
++
++

In [14]:
%%sql

SELECT * FROM nyc.long_distances

distance,total,vendor_id,pickup_date,dropoff_date
116.91,-408.85,2,2022-01-31 22:22:53,2022-02-01 00:34:41
51.16,-207.48,2,2022-01-14 19:25:02,2022-01-14 20:32:19
44.83,-144.6,2,2022-01-26 17:42:57,2022-01-26 19:27:50
53.05,-134.85,2,2022-01-07 15:37:09,2022-01-07 16:23:34
36.69,-131.6,2,2022-01-05 16:38:10,2022-01-05 18:08:35
46.67,-106.65,2,2022-01-12 21:27:20,2022-01-12 22:02:16
47.01,-101.6,2,2022-01-11 06:19:19,2022-01-11 07:03:04
45.06,-94.1,2,2022-01-28 18:08:24,2022-01-28 18:58:27
37.15,-53.55,2,2022-01-31 19:26:39,2022-01-31 20:02:41
35.9,0.3,1,2022-01-28 19:02:55,2022-01-28 19:50:20


In [15]:
%%sql

SELECT count(*) FROM nyc.long_distances

count(1)
1340


## Write another month of data

Let's write another month of data and see how the results of the view change

In [16]:
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2022-02.parquet")
df.writeTo("nyc.taxis").append()

In [17]:
%%sql

SELECT * FROM nyc.long_distances

distance,total,vendor_id,pickup_date,dropoff_date
116.91,-408.85,2,2022-01-31 22:22:53,2022-02-01 00:34:41
38.75,-286.24,2,2022-02-05 03:25:27,2022-02-05 04:26:27
75.79,-220.1,2,2022-02-17 16:05:40,2022-02-17 17:49:26
72.06,-218.85,2,2022-02-03 01:16:51,2022-02-03 02:22:09
51.16,-207.48,2,2022-01-14 19:25:02,2022-01-14 20:32:19
43.35,-202.8,2,2022-02-01 23:31:22,2022-02-02 00:24:03
47.59,-200.6,2,2022-02-25 13:07:48,2022-02-25 14:19:56
35.16,-195.6,2,2022-02-25 17:27:22,2022-02-25 18:25:38
37.14,-170.6,2,2022-02-21 16:44:09,2022-02-21 17:51:24
46.58,-158.05,2,2022-02-05 20:53:58,2022-02-05 21:42:45


In [18]:
%%sql

SELECT count(*) FROM nyc.long_distances

count(1)
2477


## Create another view
It appears that there are trips with negative total amounts. Let's display these results in a separate view

In [19]:
%%sql

CREATE OR REPLACE VIEW nyc.negative_amounts (
    total COMMENT 'Total amount',
    distance COMMENT 'Trip Distance',
    vendor_id COMMENT 'Vendor ID',
    pickup_date,
    dropoff_date)
    AS SELECT total_amount, trip_distance, VendorID, tpep_pickup_datetime, tpep_dropoff_datetime
    FROM nyc.taxis
    WHERE total_amount < 0 ORDER BY total_amount

++
||
++
++

In [20]:
%%sql

SELECT * FROM nyc.negative_amounts

total,distance,vendor_id,pickup_date,dropoff_date
-600.3,1.29,2,2022-02-03 18:35:56,2022-02-03 18:38:57
-480.3,0.0,2,2022-01-02 17:18:54,2022-01-02 17:19:06
-408.85,116.91,2,2022-01-31 22:22:53,2022-02-01 00:34:41
-358.55,6.96,2,2022-02-01 19:32:19,2022-02-01 19:49:03
-300.3,0.0,2,2022-02-25 00:52:30,2022-02-25 00:52:35
-286.24,38.75,2,2022-02-05 03:25:27,2022-02-05 04:26:27
-273.3,0.0,2,2022-02-25 16:53:35,2022-02-25 16:53:39
-262.8,0.0,2,2022-02-26 21:14:13,2022-02-26 21:17:29
-253.3,0.1,2,2022-01-19 01:39:15,2022-01-19 01:43:05
-252.8,0.0,2,2022-02-25 00:22:02,2022-02-25 00:27:40


## Listing and describing views

In [21]:
%%sql

SHOW VIEWS in nyc

namespace,viewName,isTemporary
nyc,long_distances,False
nyc,negative_amounts,False


In [22]:
%%sql

SHOW VIEWS in nyc LIKE '*neg*'

namespace,viewName,isTemporary
nyc,negative_amounts,False


In [23]:
%%sql

DESCRIBE nyc.long_distances

col_name,data_type,comment
distance,double,Trip Distance
total,double,Total amount
vendor_id,bigint,Vendor ID
pickup_date,timestamp,
dropoff_date,timestamp,


In [24]:
%%sql

DESCRIBE EXTENDED nyc.long_distances

col_name,data_type,comment
distance,double,Trip Distance
total,double,Total amount
vendor_id,bigint,Vendor ID
pickup_date,timestamp,
dropoff_date,timestamp,
,,
# Detailed View Information,,
Comment,,
View Catalog and Namespace,demo.nyc,
View Query Output Columns,"[trip_distance, total_amount, VendorID, tpep_pickup_datetime, tpep_dropoff_datetime]",


## Displaying the CREATE statement of a view

In [25]:
%%sql

SHOW CREATE TABLE nyc.long_distances

createtab_stmt
"CREATE VIEW demo.nyc.long_distances ( distance COMMENT 'Trip Distance', total COMMENT 'Total amount', vendor_id COMMENT 'Vendor ID', pickup_date, dropoff_date)TBLPROPERTIES ( 'format-version' = '1', 'location' = 's3://warehouse/nyc/long_distances', 'provider' = 'iceberg')ASSELECT trip_distance, total_amount, VendorID, tpep_pickup_datetime, tpep_dropoff_datetime FROM nyc.taxis WHERE trip_distance > 35 ORDER BY total_amount, trip_distance"


## Altering and displaying properties of a view

This will add a new property and also update the comment of the view. 
The comment will be shown when describing the view.
The end of this section will also remove a property from the view.

In [27]:
%%sql

SHOW TBLPROPERTIES nyc.long_distances

key,value
location,s3://warehouse/nyc/long_distances
provider,iceberg
format-version,1


In [28]:
%%sql

ALTER VIEW nyc.long_distances SET TBLPROPERTIES ('key1' = 'val1', 'key2' = 'val2', 'comment' = 'This is a view comment')

++
||
++
++

In [29]:
%%sql

SHOW TBLPROPERTIES nyc.long_distances

key,value
location,s3://warehouse/nyc/long_distances
provider,iceberg
key1,val1
key2,val2
format-version,1


In [30]:
%%sql

DESCRIBE EXTENDED nyc.long_distances

col_name,data_type,comment
distance,double,Trip Distance
total,double,Total amount
vendor_id,bigint,Vendor ID
pickup_date,timestamp,
dropoff_date,timestamp,
,,
# Detailed View Information,,
Comment,This is a view comment,
View Catalog and Namespace,demo.nyc,
View Query Output Columns,"[trip_distance, total_amount, VendorID, tpep_pickup_datetime, tpep_dropoff_datetime]",


In [31]:
%%sql

ALTER VIEW nyc.long_distances UNSET TBLPROPERTIES ('key1')

++
||
++
++

In [32]:
%%sql

SHOW TBLPROPERTIES nyc.long_distances

key,value
location,s3://warehouse/nyc/long_distances
provider,iceberg
key2,val2
format-version,1
